In [1]:
import $ivy.`org.typelevel::cats:0.9.0`

import $ivy.$                          

# Daily Processing
* [Download Updates to Account](#(Download-Updates-to-Account)
    * [Download OFX Statement from Bank](#Download-OFX-Statement-from-Bank)
    * [Remove Existing Transactions](#Remove-Existing-Transactions)
    * [Update Transactions with Payee](#Update-Transactions-with-Payee)
    * [Update Database](#Update-Database)
* [Mark Budget Item Paid](#Mark-Budget-Item-Paid)
* [Generate Budget Report](#Generate-Budget-Report)
* [Send Notification](#Send-Notification)


## Download Updates to Account
---
### Download OFX Statement from Bank
- Get bank ofxid, ofxurl, ofxapp, ofxappversion from Banks
- Download OFX statement using ofx4j
- Extract Balance and Transactions from OFX Statement
    - Balance Information
    - As of Date
    - Current Balance
- List of Transactions (since last update?)
- Update Account table with Balance Information

In [2]:
import cats.free.Free
import cats.free.Free.liftF

case class Balance(asOf: java.util.Date, balance: Double)
case class Account(id: Long, name: String, currentBalance: Balance)
case class Payee(name: String, parent: Option[Payee] = None)
case class Transaction(id: Long, account: Account, posted: java.util.Date, name: String, amount: Double)
case class AccountUpdate(account: Account, balance: Balance, transactions: List[Transaction])

sealed trait Command[T]
case class DownloadUpdate(acct: Account) extends Command[AccountUpdate]
case class MergeTransactions(txns: List[Transaction]) extends Command[Unit]
case class UpdateBalance(acct: Account, balance: Balance) extends Command[Unit]


import cats.free.Free

import cats.free.Free.liftF


defined class Balance
defined class Account
defined class Payee
defined class Transaction
defined class AccountUpdate
defined trait Command
defined class DownloadUpdate
defined class MergeTransactions
defined class UpdateBalance

### Remove Existing Transactions
- Get existing transactions based on account id and fitid
- Remove these transactions from list of transactions
- Insert newly created Transactions

### Update Transactions with Payee
- Use transaction name to match payee aliases from database
- Add Payee and Payee Alias for any missing name -> alias pair
- Update Transaction with payee id

In [4]:
import cats.arrow.FunctionK
import cats.{Id, ~>}

type FreeCommand[T] = Free[Command, T]

def downloadUpdate(acct: Account): FreeCommand[AccountUpdate] =
  liftF[Command, AccountUpdate](DownloadUpdate(acct))

def mergeTransactions(txn: List[Transaction]): FreeCommand[Unit] =
  liftF[Command, Unit](MergeTransactions(txn))

def updateBalance(acct: Account, balance: Balance): FreeCommand[Unit] =
  liftF[Command, Unit](UpdateBalance(acct, balance))

def program(account: Account): FreeCommand[AccountUpdate] = for {
  dl <- downloadUpdate(account)
  _ <- mergeTransactions(dl.transactions)
  _ <- updateBalance(account, dl.balance)
  } yield dl

def compiler: Command ~> Id = new (Command ~> Id) {
  def apply[A](fa: Command[A]): Id[A] = fa match {
  case DownloadUpdate(acct) =>
    println(s"Downloading account update for $acct")
    AccountUpdate(acct, Balance(new java.util.Date(), 0.9), List())
  case MergeTransactions(txns) =>
    println(s"Merge transactions: $txns")
    ()
  case UpdateBalance(acct, balance) =>
    println(s"Update balance for $acct = $balance")
    ()
  }
}

val acct = Account(0l, "test", Balance(new java.util.Date(), 0.0))
val result = program(acct).foldMap(compiler)


import cats.arrow.FunctionK

import cats.{Id, ~>}


defined type FreeCommand
defined function downloadUpdate
defined function mergeTransactions
defined function updateBalance
defined function program
defined function compiler
acct: Account = Account(0,test,Balance(Tue Apr 04 07:14:50 EDT 2017,0.0))
result: Id[AccountUpdate] = AccountUpdate(Account(0,test,Balance(Tue Apr 04 07:14:50 EDT 2017,0.0)),Balance(Tue Apr 04 07:14:50 EDT 2017,0.9),List())

Merge transactions: List()
Update balance for Account(0,test,Balance(Tue Apr 04 07:14:50 EDT 2017,0.0)) = Balance(Tue Apr 04 07:14:50 EDT 2017,0.9)


## Mark Budget Item Paid
---
- Get unpaid budget items for period, usually EOM
    - Create data range around item's due date (+/- 7 days)
    - Find first transaction in range
    - Update item as paid

## Generate Budget Report
---

## Send Notification
---